교재 예제처럼 spark.read로 csv를 불러오려다 무한 오류에 빠졌었습니다. (ec2)

그래서 pandas로 받아와서 DataFrame으로 변환하게 되었습니다.

In [ ]:
# ms 제공 예제 : pd -> df
# Generate a pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

In [5]:
spark

In [2]:
myRange = spark.range(1000).toDF('number')
# 1개의 컬럼과 1000개의 로우. 각 로우에 0~999까지의 값이 할당
# 각 부분이 서로 다른 익스큐터에 할당됨

In [3]:
divisBy2 = myRange.where('number % 2 = 0')
# 액션을 하지 않아 결과 출력 x
# 짝수 찾는 예제

In [4]:
# 액션
divisBy2.count()

500

In [1]:
pwd

'/home/ubuntu/hadoop-2.9.0/spark-defintive-guide'

In [3]:
ls ./flight-data

2010-summary.csv  2012-summary.csv  2014-summary.csv
2011-summary.csv  2013-summary.csv  2015-summary.csv


In [1]:
pip install pyarrow

     |████████████████████████████████| 22.6MB 4.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyarrow
import pandas as pd
from pyarrow import csv

In [ ]:
# 실패한 코드

flightData2015 = spark\
.read\
.option('inferSchema', 'true')\
.option('header', 'true')\
.csv('./flight-data/2015-summary.csv')

In [14]:
flightData2015pandas = pd.read_csv('./flight-data/2015-summary.csv')
flightData2015 = spark.createDataFrame(flight)

In [10]:
print(df)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]


In [12]:
# head와 동일
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [15]:
flightData2015.sort('count').explain()

== Physical Plan ==
*(1) Sort [count#14L ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#14L ASC NULLS FIRST, 200)
   +- Scan ExistingRDD[DEST_COUNTRY_NAME#12,ORIGIN_COUNTRY_NAME#13,count#14L]


In [17]:
spark.conf.set('spark.sql.shuffle.partitions', '5')
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='Burkina Faso', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='Djibouti', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [19]:
# 모든 df를 테이블이나 뷰로 만듦
# SQL 사용 가능
flightData2015.createOrReplaceTempView('flightData2015')

***spark.sql 사용하는 과정에서 지속적으로 에러. 원인 찾지 못함.***

In [24]:
# 두 가지 모두 동일한 실행 계획으로 컴파일 됨

# sqlWay = spark.sql("""
# SELECT DEST_COUNTRY_NAME, count(1)
# FROM flightData2015
# GROUP BY DEST_COUNTRY_NAME
# """)

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

# sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#12, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[partial_count(1)])
      +- *(1) Project [DEST_COUNTRY_NAME#12]
         +- Scan ExistingRDD[DEST_COUNTRY_NAME#12,ORIGIN_COUNTRY_NAME#13,count#14L]


In [22]:
from pyspark.sql.functions import max

flightData2015.select(max('count')).take(1)

[Row(max(count)=370002)]

In [27]:
# 둘 모두 동일한 결과

# maxSql = spark.sql("""
# SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
# FROM flightData2015
# GROUP BY DEST_COUNTRY_NAME
# ORDER BY sum(count) DESC
# LIMIT 5
# """)
# maxSQL.show()

from pyspark.sql.functions import desc
flightData2015\
.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)', 'destination_total')\
.sort(desc('destination_total'))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [28]:
flightData2015\
.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)', 'destination_total')\
.sort(desc('destination_total'))\
.limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#68L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#12,destination_total#68L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[sum(count#14L)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#12, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[partial_sum(count#14L)])
         +- *(1) Project [DEST_COUNTRY_NAME#12, count#14L]
            +- Scan ExistingRDD[DEST_COUNTRY_NAME#12,ORIGIN_COUNTRY_NAME#13,count#14L]
